# Artificial Neural Network for prediciting Camber and Toe extrema

## Install any missing libraries

In [160]:
!pip3 install openpyxl

## Import all libraries

In [1]:
import glob 
import random
import numpy as np
import pandas as pd
import math
import os
import copy
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

## Importing, Cleaning, and Preparing Data

In [2]:
files=glob.glob("Population/*.xlsx",recursive=True)
population=[]
for i in range(0, len(files)):
    parent=pd.read_excel(files[i],header=None).values
    population.append(parent)
population=np.array(population,dtype=object)

In [3]:
all_geometry = []
all_results = []
for i in range(0, len(population)):
    pop = pd.DataFrame(population[i])
    pop = pop.drop(index=[20, 21], axis=0)
    geometry = pop.iloc[14:22, 1:4].values
    results = pop.iloc[82:96, 1:6].values
    all_geometry.append(geometry)
    all_results.append(results)
all_geometry = np.array(all_geometry)
all_results = np.array(all_results)

In [4]:
X_flat=all_geometry.reshape((all_geometry.shape[0], -1))

In [5]:
X=[]
output=[]
for i in range(X_flat.shape[0]):
    for j in range(all_results[i].shape[0]):
        X.append(list(np.append(X_flat[i],all_results[i][j][0])))
        output.append(list(all_results[i][j][1:]))
X=np.array(X)
output=np.array(output)

In [4]:
#output=np.empty(shape=[all_results.shape[0],4])
#for i in range(all_results.shape[0]):
    #output[i,0]=max(all_results[i,:,0])
    #output[i,1]=min(all_results[i,:,0])
    #output[i,2]=max(all_results[i,:,1])
    #output[i,3]=min(all_results[i,:,1])

In [6]:
y=output
X=X.astype('float32')
y=y.astype('float32')

#X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

train_df=pd.DataFrame(X)
train_df[25]=y[:,0]
train_df[26]=y[:,1]
train_df[27]=y[:,2]
train_df[28]=y[:,3]
target=train_df[24:28]
train_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,3939.0,-180.0,485.000000,4259.0,-180.000000,455.0,4082.000000,-464.5,255.000000,4072.5,...,-490.500000,311.00000,4050.5,-208.0,522.000000,200.0,-1.0623,-0.6866,7.8543,4.7990
1,3939.0,-180.0,485.000000,4259.0,-180.000000,455.0,4082.000000,-464.5,255.000000,4072.5,...,-490.500000,311.00000,4050.5,-208.0,522.000000,180.0,-0.8601,-0.5627,7.6417,4.6140
2,3939.0,-180.0,485.000000,4259.0,-180.000000,455.0,4082.000000,-464.5,255.000000,4072.5,...,-490.500000,311.00000,4050.5,-208.0,522.000000,160.0,-0.6782,-0.4570,7.4288,4.4460
3,3939.0,-180.0,485.000000,4259.0,-180.000000,455.0,4082.000000,-464.5,255.000000,4072.5,...,-490.500000,311.00000,4050.5,-208.0,522.000000,140.0,-0.5157,-0.3677,7.2155,4.2947
4,3939.0,-180.0,485.000000,4259.0,-180.000000,455.0,4082.000000,-464.5,255.000000,4072.5,...,-490.500000,311.00000,4050.5,-208.0,522.000000,120.0,-0.3725,-0.2932,7.0019,4.1601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,3941.5,-236.0,495.420013,4249.5,-237.800003,465.0,4075.120117,-531.5,269.350006,3942.5,...,-548.900024,336.23999,3969.5,-244.0,571.799988,20.0,0.1856,0.4697,4.5553,4.9605
528,3941.5,-236.0,495.420013,4249.5,-237.800003,465.0,4075.120117,-531.5,269.350006,3942.5,...,-548.900024,336.23999,3969.5,-244.0,571.799988,0.0,0.0000,0.0000,4.6180,5.1090
529,3941.5,-236.0,495.420013,4249.5,-237.800003,465.0,4075.120117,-531.5,269.350006,3942.5,...,-548.900024,336.23999,3969.5,-244.0,571.799988,-20.0,-0.2582,-0.2935,4.6737,5.3437
530,3941.5,-236.0,495.420013,4249.5,-237.800003,465.0,4075.120117,-531.5,269.350006,3942.5,...,-548.900024,336.23999,3969.5,-244.0,571.799988,-40.0,-0.6288,-0.3084,4.7212,5.7131


## Scaling Data between 0 and 1

In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train_df)

# Print out the adjustment that the scaler applied to the data
print("Note: Median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[28], scaler.min_[28]))
multiplied_by = scaler.scale_[28]
added = scaler.min_[28]

scaled_train_df = pd.DataFrame(scaled_train, columns=train_df.columns.values)
scaled_train_df

Note: Median values were scaled by multiplying by 0.0407901853 and adding 0.238794


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,0.947369,0.651163,0.166667,1.000000,0.173142,0.594970,0.470351,0.434546
1,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,0.947369,0.651163,0.166667,0.923077,0.181996,0.596000,0.464545,0.427000
2,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,0.947369,0.651163,0.166667,0.846154,0.189960,0.596880,0.458731,0.420147
3,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,0.947369,0.651163,0.166667,0.769231,0.197075,0.597623,0.452905,0.413976
4,0.519482,0.103310,0.638569,0.800003,0.983079,0.500000,0.908432,0.985294,0.582242,1.000000,...,0.660634,0.162920,0.947369,0.651163,0.166667,0.692308,0.203345,0.598243,0.447072,0.408485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,0.530304,0.001455,0.971265,0.610001,0.005076,0.666667,0.593414,0.000000,1.000000,0.071428,...,0.000000,0.574128,0.094738,0.232558,0.996667,0.307692,0.227781,0.604590,0.380255,0.441134
528,0.530304,0.001455,0.971265,0.610001,0.005076,0.666667,0.593414,0.000000,1.000000,0.071428,...,0.000000,0.574128,0.094738,0.232558,0.996667,0.230769,0.219655,0.600682,0.381968,0.447191
529,0.530304,0.001455,0.971265,0.610001,0.005076,0.666667,0.593414,0.000000,1.000000,0.071428,...,0.000000,0.574128,0.094738,0.232558,0.996667,0.153846,0.208350,0.598240,0.383489,0.456764
530,0.530304,0.001455,0.971265,0.610001,0.005076,0.666667,0.593414,0.000000,1.000000,0.071428,...,0.000000,0.574128,0.094738,0.232558,0.996667,0.076923,0.192123,0.598116,0.384786,0.471832


## ANN Model

In [7]:
model = Sequential()

model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(4))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [10]:
X = scaled_train_df[scaled_train_df.columns[:25]].values
Y = scaled_train_df[scaled_train_df.columns[25:29]].values

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.3)

In [12]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=10,
    shuffle=False,
    verbose=1
)

Epoch 1/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0281 - accuracy: 0.9194
Epoch 2/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0066 - accuracy: 0.9731
Epoch 3/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0047 - accuracy: 0.9758
Epoch 4/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 0.9758
Epoch 5/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0036 - accuracy: 0.9731
Epoch 6/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0032 - accuracy: 0.9731
Epoch 7/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 0.9704
Epoch 8/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 0.9731
Epoch 9/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 0.9731
Epoch 10/1000
38/38 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 0.9812

38/38 [==============================] - 0s 1ms/step - loss: 1.3200e-04 - accuracy: 0.9892
Epoch 81/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.2651e-04 - accuracy: 0.9919
Epoch 82/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.1960e-04 - accuracy: 0.9919
Epoch 83/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.1472e-04 - accuracy: 0.9919
Epoch 84/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.0915e-04 - accuracy: 0.9919
Epoch 85/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.0519e-04 - accuracy: 0.9919
Epoch 86/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.0089e-04 - accuracy: 0.9919
Epoch 87/1000
38/38 [==============================] - 0s 2ms/step - loss: 9.3774e-05 - accuracy: 0.9946
Epoch 88/1000
38/38 [==============================] - 0s 2ms/step - loss: 9.1250e-05 - accuracy: 0.9946
Epoch 89/1000
38/38 [==============================] - 0s 1ms/step - 

38/38 [==============================] - 0s 1ms/step - loss: 1.1910e-04 - accuracy: 0.9946
Epoch 158/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.1048e-04 - accuracy: 0.9892
Epoch 159/1000
38/38 [==============================] - 0s 1ms/step - loss: 8.7051e-05 - accuracy: 0.9946
Epoch 160/1000
38/38 [==============================] - 0s 1ms/step - loss: 7.1180e-05 - accuracy: 0.9946
Epoch 161/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.4578e-05 - accuracy: 0.9946
Epoch 162/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.2142e-05 - accuracy: 0.9946
Epoch 163/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.5299e-05 - accuracy: 0.9946
Epoch 164/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.9754e-05 - accuracy: 0.9946
Epoch 165/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.6429e-05 - accuracy: 0.9946
Epoch 166/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 3.5433e-05 - accuracy: 0.9946
Epoch 235/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.5625e-05 - accuracy: 0.9946
Epoch 236/1000
38/38 [==============================] - 0s 2ms/step - loss: 3.6243e-05 - accuracy: 0.9946
Epoch 237/1000
38/38 [==============================] - 0s 2ms/step - loss: 3.5025e-05 - accuracy: 0.9946
Epoch 238/1000
38/38 [==============================] - 0s 2ms/step - loss: 3.6151e-05 - accuracy: 0.9946
Epoch 239/1000
38/38 [==============================] - 0s 2ms/step - loss: 3.6910e-05 - accuracy: 0.9946
Epoch 240/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.6072e-05 - accuracy: 0.9973
Epoch 241/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.0524e-05 - accuracy: 0.9946
Epoch 242/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.6376e-05 - accuracy: 0.9973
Epoch 243/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 8.0084e-05 - accuracy: 0.9973
Epoch 312/1000
38/38 [==============================] - 0s 1ms/step - loss: 8.3745e-05 - accuracy: 0.9946
Epoch 313/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.9795e-05 - accuracy: 0.9973
Epoch 314/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.9629e-05 - accuracy: 0.9946
Epoch 315/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.1074e-05 - accuracy: 0.9946
Epoch 316/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.4117e-05 - accuracy: 0.9946
Epoch 317/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.5323e-05 - accuracy: 0.9946
Epoch 318/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.3703e-05 - accuracy: 0.9946
Epoch 319/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.7928e-05 - accuracy: 0.9973
Epoch 320/1000
38/38 [==============================] - 0s 1m

...

38/38 [==============================] - 0s 1ms/step - loss: 1.5839e-05 - accuracy: 0.9946
Epoch 620/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.5294e-05 - accuracy: 0.9973
Epoch 621/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.4563e-05 - accuracy: 0.9946
Epoch 622/1000
38/38 [==============================] - 0s 2ms/step - loss: 1.4864e-05 - accuracy: 0.9973
Epoch 623/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.4904e-05 - accuracy: 0.9946
Epoch 624/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.5288e-05 - accuracy: 0.9973
Epoch 625/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.4625e-05 - accuracy: 0.9946
Epoch 626/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.5590e-05 - accuracy: 0.9973
Epoch 627/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.6442e-05 - accuracy: 0.9946
Epoch 628/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 3.1982e-05 - accuracy: 0.9946
Epoch 697/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.0334e-05 - accuracy: 0.9946
Epoch 698/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.7551e-05 - accuracy: 0.9946
Epoch 699/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.5211e-05 - accuracy: 0.9946
Epoch 700/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.9788e-05 - accuracy: 0.9973
Epoch 701/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.0915e-05 - accuracy: 0.9946
Epoch 702/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.1094e-05 - accuracy: 0.9946
Epoch 703/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.1195e-05 - accuracy: 0.9892
Epoch 704/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.7545e-05 - accuracy: 0.9919
Epoch 705/1000
38/38 [==============================] - 0s 1m

Epoch 773/1000
38/38 [==============================] - 0s 2ms/step - loss: 9.4786e-04 - accuracy: 0.9919
Epoch 774/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.3134e-04 - accuracy: 0.9946
Epoch 775/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.6948e-04 - accuracy: 0.9919
Epoch 776/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.7978e-04 - accuracy: 0.9946
Epoch 777/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.7795e-04 - accuracy: 0.9946
Epoch 778/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.2862e-04 - accuracy: 0.9919
Epoch 779/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.2721e-04 - accuracy: 0.9946
Epoch 780/1000
38/38 [==============================] - 0s 1ms/step - loss: 6.6816e-05 - accuracy: 0.9946
Epoch 781/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.9263e-05 - accuracy: 0.9946
Epoch 782/1000
38/38 [========================

38/38 [==============================] - 0s 1ms/step - loss: 1.5935e-05 - accuracy: 0.9946
Epoch 851/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.1832e-05 - accuracy: 0.9973
Epoch 852/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.0967e-05 - accuracy: 0.9946
Epoch 853/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.9862e-05 - accuracy: 0.9946
Epoch 854/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.6165e-05 - accuracy: 0.9946
Epoch 855/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.1885e-05 - accuracy: 0.9973
Epoch 856/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.9145e-05 - accuracy: 0.9973
Epoch 857/1000
38/38 [==============================] - 0s 1ms/step - loss: 2.3714e-05 - accuracy: 0.9946
Epoch 858/1000
38/38 [==============================] - 0s 1ms/step - loss: 1.7466e-05 - accuracy: 0.9946
Epoch 859/1000
38/38 [==============================] - 0s 1m

38/38 [==============================] - 0s 1ms/step - loss: 6.9861e-05 - accuracy: 0.9946
Epoch 928/1000
38/38 [==============================] - 0s 1ms/step - loss: 7.0565e-05 - accuracy: 0.9919
Epoch 929/1000
38/38 [==============================] - 0s 1ms/step - loss: 7.1752e-05 - accuracy: 0.9973
Epoch 930/1000
38/38 [==============================] - 0s 1ms/step - loss: 9.4751e-05 - accuracy: 0.9919
Epoch 931/1000
38/38 [==============================] - 0s 2ms/step - loss: 5.6041e-05 - accuracy: 0.9892
Epoch 932/1000
38/38 [==============================] - 0s 2ms/step - loss: 5.3576e-05 - accuracy: 0.9946
Epoch 933/1000
38/38 [==============================] - 0s 1ms/step - loss: 4.2095e-05 - accuracy: 0.9946
Epoch 934/1000
38/38 [==============================] - 0s 1ms/step - loss: 5.4611e-05 - accuracy: 0.9973
Epoch 935/1000
38/38 [==============================] - 0s 1ms/step - loss: 3.5081e-05 - accuracy: 0.9973
Epoch 936/1000
38/38 [==============================] - 0s 1m

## Predictions

In [13]:
prediction = model.predict(X_train[:5])
print('Prediction with scaling - ',format(prediction))

y_pred = prediction
y_pred -= added
y_pred /= multiplied_by
print("Prediction - ",format(y_pred))

Prediction with scaling -  [[0.21761635 0.5967636  0.40176547 0.3938002 ]
 [0.20183986 0.59418714 0.43679783 0.40575063]
 [0.20228364 0.61714613 0.42061618 0.45850214]
 [0.21518166 0.60449004 0.40459186 0.38673663]
 [0.20824262 0.5957346  0.43148494 0.40078682]]
Prediction -  [[-0.5191824   8.775878    3.9953623   3.8000882 ]
 [-0.90595394  8.712714    4.854205    4.0930614 ]
 [-0.8950746   9.27557     4.457501    5.386302  ]
 [-0.5788704   8.965297    4.0646534   3.62692   ]
 [-0.7489858   8.750651    4.723956    3.97137   ]]


In [14]:
y_train[:5]

array([[0.22227329, 0.60140437, 0.4058476 , 0.39221394],
       [0.2087657 , 0.5987544 , 0.44123036, 0.4037127 ],
       [0.20553876, 0.6202038 , 0.42662776, 0.45870602],
       [0.21493496, 0.6073042 , 0.40787947, 0.38780046],
       [0.21328866, 0.5991754 , 0.4353751 , 0.39970303]], dtype=float32)

In [15]:
Y_pred=model.predict(X_test)

In [16]:
Y_pred

array([[ 0.21570255,  0.6118059 ,  0.41393286,  0.40696257],
       [ 0.19864267,  0.5851133 ,  0.3814497 ,  0.40697944],
       [ 0.17389704,  0.59920335,  0.46431306,  0.43451852],
       [ 0.19545005,  0.5933014 ,  0.4418054 ,  0.411482  ],
       [ 0.17785603,  0.59317887,  0.45559838,  0.43013635],
       [ 0.20248175,  0.59024453,  0.4337463 ,  0.40020987],
       [ 0.3022837 ,  0.5484953 ,  0.36645234,  0.31685948],
       [ 0.20475097,  0.59489095,  0.42969304,  0.40390447],
       [ 0.21847275,  0.597903  ,  0.40798306,  0.39613098],
       [ 0.17868039,  0.59244895,  0.44886193,  0.42727077],
       [ 0.1953639 ,  0.6024871 ,  0.3987977 ,  0.40862095],
       [ 0.20190156,  0.5925641 ,  0.45264944,  0.40542066],
       [ 0.21689832,  0.61084235,  0.4132735 ,  0.40389574],
       [ 0.17055413,  0.5930518 ,  0.46771708,  0.4353818 ],
       [ 0.22833242,  0.6500746 ,  0.3604966 ,  0.44722986],
       [ 0.20746076,  0.59733605,  0.40933332,  0.38842365],
       [ 0.21185276,  0.

In [17]:
y_test

array([[0.21288584, 0.6292436 , 0.42284805, 0.41609251],
       [0.20452295, 0.58919454, 0.38989586, 0.40347612],
       [0.18035376, 0.5991721 , 0.46452573, 0.43060565],
       [0.20457111, 0.5986804 , 0.44838828, 0.40761632],
       [0.18654056, 0.59681666, 0.45868415, 0.42329198],
       [0.21117823, 0.595106  , 0.4294352 , 0.3994175 ],
       [0.30324882, 0.55188113, 0.37121585, 0.32024372],
       [0.20003064, 0.5939703 , 0.42162183, 0.4092765 ],
       [0.21965496, 0.6006822 , 0.41180664, 0.39437175],
       [0.18996014, 0.5968799 , 0.45873058, 0.42014706],
       [0.19098033, 0.5918021 , 0.4143055 , 0.40260726],
       [0.20321378, 0.5975522 , 0.45864862, 0.4080283 ],
       [0.21739566, 0.6159663 , 0.419153  , 0.40447956],
       [0.18296334, 0.5977693 , 0.4803928 , 0.4272282 ],
       [0.24268137, 0.653546  , 0.36450854, 0.44483328],
       [0.20223738, 0.599326  , 0.41327044, 0.39583203],
       [0.20834973, 0.59824026, 0.38348895, 0.4567644 ],
       [0.20242128, 0.602615  ,